In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

In [3]:
database_type = "/demo"

The `intakeOutput` table provides Intake and output recorded for patients entered from the nursing flowsheet.
See: http://eicu-crd.mit.edu/eicutables/intakeOutput/

Urine output is measured in mL/kg.
Urine output in mL/ Patient weight in kg.

#### Reading in lab dataset and keeping relevant columns to our study .

In [4]:
columns = ['patientunitstayid', 'intakeoutputoffset', 'intaketotal', 'outputtotal', 'celllabel']
intakeOutput = pd.read_csv('../../eICU' + database_type + '/intakeOutput.csv', usecols=columns)
intakeOutput

,patientunitstayid,intakeoutputoffset,intaketotal,outputtotal,celllabel
0,147307,-394,0.00,0.0,Bodyweight (lb)
1,147307,-394,0.00,0.0,Bodyweight (kg)
2,211715,1533,120.00,0.0,P.O.
3,219981,6504,120.00,0.0,P.O.
4,158057,624,0.00,0.0,Bodyweight (lb)
...,...,...,...,...,...
100461,3348409,1930,68.75,0.0,Crystalloids
100462,3329305,1366,150.00,150.0,Crystalloids
100463,3346866,2980,480.00,350.0,P.O.
100464,3346866,2980,480.00,350.0,Urine


#### Reading in cleaned patient data

In [5]:
patient = pd.read_csv('../../eICU/training/patient.csv')
patient

,patientunitstayid,admissionweight
0,141764,166.5
1,141765,46.5
2,143870,77.5
3,144815,60.3
4,145427,91.7
...,...,...
2515,3351763,134.5
2516,3352230,127.0
2517,3352231,127.0
2518,3352333,68.3


#### Filtering out reading pertaining to urine output 

In [6]:
intakeOutputUrine = intakeOutput.loc[(intakeOutput['celllabel'].str.contains("Urine"))]
intakeOutputUrine

,patientunitstayid,intakeoutputoffset,intaketotal,outputtotal,celllabel
11,176504,1239,771.02,150.0,Urine
18,144815,773,0.00,700.0,Urine
20,239655,3010,0.00,300.0,Urine
21,167186,-821,0.00,400.0,Urine
32,210641,306,0.00,200.0,Urine
...,...,...,...,...,...
100448,3352333,900,75.00,45.0,Urine
100453,3329305,1366,150.00,150.0,Urine
100455,3334456,2027,125.00,70.0,Urine
100459,3348409,1815,125.00,125.0,Urine


#### Merging on patient table to get patient weight

In [7]:
intakeOutputUrine = pd.DataFrame.merge(patient, intakeOutputUrine, on='patientunitstayid')

#### Calculating patient urine output by diving urine output in mL by the patient weight

In [8]:
intakeOutputUrine['urineoutputbyweight'] = intakeOutputUrine.apply(lambda x: x['outputtotal'] / x['admissionweight'], axis=1)

#### Dropping irrelevant columns to study.

In [9]:
intakeOutputUrine = intakeOutputUrine[['patientunitstayid','intakeoutputoffset', 'urineoutputbyweight']]
intakeOutputUrine.drop_duplicates()
intakeOutputUrine

,patientunitstayid,intakeoutputoffset,urineoutputbyweight
0,141764,-14,3.003003
1,141765,1401,12.903226
2,141765,556,10.752688
3,141765,1936,10.752688
4,141765,2056,11.827957
...,...,...,...
33197,3353113,1869,2.556818
33198,3353113,1089,0.284091
33199,3353113,2169,1.136364
33200,3353113,2109,1.363636


#### Saving the intakeOutputUrine DataFrame to a `csv` file

In [10]:
path = "../../eICU/training/"
intakeOutputUrine.to_csv(path + "intakeOutputUrine.csv", sep=',', index=False, encoding='utf-8')
